https://aws.amazon.com/blogs/machine-learning/create-and-fine-tune-sentence-transformers-for-enhanced-classification-accuracy/

In [2]:
import numpy as np
import torch
import torchmetrics
from sentence_transformers import SentenceTransformer
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import re
from torch import nn, optim
import torchmetrics
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, Specificity, AUROC, ROC

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [81]:
# 1. Load a pretrained Sentence Transformer model
# model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
# embedding_size = 384


In [82]:
# print(f"Max seq len = {model.max_seq_length}")
# sentences = ["This is sentence 1.",
#              "This is sentence 2."]

# embeddings = model.encode(sentences)
# print(type(embeddings))
# print(embeddings.shape)

# similarities = model.similarity(embeddings, embeddings)
# print(similarities)

Max seq len = 256
<class 'numpy.ndarray'>
(2, 384)
tensor([[1.0000, 0.9287],
        [0.9287, 1.0000]])


In [83]:
model.prompts
max seq len and truncation

{}

In [3]:
num_classes = 9
# embedding_size = model.get_sentence_embedding_dimension()
embedding_size = 384

class ClassificationHead(nn.Module):
    def __init__(self, embedding_size: int, num_classes: int):
        super(ClassificationHead, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(embedding_size, num_classes),
        )
    
    def forward(self, features: torch.Tensor):
        x = features["sentence_embedding"]
        x = self.layers(x)
        return x

class SentenceTransformerWithHead(nn.Module):
    def __init__(self, transformer: SentenceTransformer, head: ClassificationHead):
        super(SentenceTransformerWithHead, self).__init__()
        self.transformer = transformer
        self.head = head
    
    def forward(self, input: dict[torch.Tensor, torch.Tensor]):
        # input['input_ids']
        # input['attention_mask']
        features = self.transformer(input)
        logits = self.head(features)
        return logits

# head = ClassificationHead(embedding_size, num_classes)
# model_with_head = SentenceTransformerWithHead(model, head)
# head.to(device)
# model_with_head.to(device)
        

In [85]:
# print(next(model.parameters()).device)
# print(next(head.parameters()).device)
# print(next(model_with_head.parameters()).device)

cuda:0
cuda:0
cuda:0


In [46]:
# inputs = model.tokenize(sentences)
# inputs = {key: value.to(device) for key, value in inputs.items()}
# inputs

{'input_ids': tensor([[ 101, 2023, 2003, 6251, 1015, 1012,  102],
         [ 101, 2023, 2003, 6251, 1016, 1012,  102]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [47]:
# logits = model_with_head(inputs)

In [35]:
class QueryDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
        self.df = self.df.astype({"Label": "int64"})
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        item = {
            "Query": row["Query"],
            "Label": row["Label"],

        }
        return item

ds = QueryDataset("intent_classifier_samples.csv")

In [36]:
ds.df.dtypes

Query       object
Label        int64
Notes       object
Category    object
dtype: object

In [37]:
train_dataloader = DataLoader(ds, batch_size=16, shuffle=True)
sample = next(iter(train_dataloader))

In [ ]:
# sample["Query"]
# sample["Label"]
# model.tokenize(sample["Query"])

In [61]:
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
head = ClassificationHead(embedding_size, num_classes)
model_with_head = SentenceTransformerWithHead(model, head)
head.to(device)
model_with_head.to(device)

SentenceTransformerWithHead(
  (transformer): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
    (2): Normalize()
  )
  (head): ClassificationHead(
    (layers): Sequential(
      (0): Linear(in_features=384, out_features=9, bias=True)
    )
  )
)

In [62]:
# Freeze all weights except classifier head
for name, param in model_with_head.named_parameters():
    # print(name)
    if not re.search(r"^head", name):
        # print(name)
        param.requires_grad = False

for name, param in model_with_head.named_parameters():
    if param.requires_grad:
        print(name)

head.layers.0.weight
head.layers.0.bias


In [63]:
loss_function = nn.CrossEntropyLoss(reduction="sum")
optimizer = optim.AdamW(model_with_head.parameters(), lr=0.01) # 0.01 for head only, 0.0001 for all weights
# lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.98)

f1 = F1Score(task="multiclass", num_classes=num_classes, average=None)
accuracy = Accuracy(task="multiclass", num_classes=num_classes, average="macro")
accuracy.to(device)

n_epochs = 20
for edx in range(n_epochs):
    cum_loss = 0
    accuracy.reset()
    for bdx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        inputs = model.tokenize(batch["Query"])
        inputs = {key: value.to(device) for key, value in inputs.items()}
        labels = batch["Label"].to(device)

        logits = model_with_head(inputs)

        loss = loss_function(logits, labels)
        cum_loss += loss.item()
        loss.backward()
        optimizer.step()
        # lr_scheduler.step()

        accuracy.update(torch.argmax(logits, dim=1), labels)
    
    cum_loss /= len(train_dataloader.dataset.df)
    # accuracy.compute()
    print(f"Loss: {cum_loss:.2f}, Accuracy: {accuracy.compute().detach().cpu().numpy().item():.2f}")

Loss: 1.18, Accuracy: 0.12
Loss: 0.66, Accuracy: 0.40
Loss: 0.48, Accuracy: 0.77
Loss: 0.37, Accuracy: 0.91
Loss: 0.30, Accuracy: 0.92
Loss: 0.24, Accuracy: 0.96
Loss: 0.21, Accuracy: 0.97
Loss: 0.18, Accuracy: 0.98
Loss: 0.15, Accuracy: 0.98
Loss: 0.14, Accuracy: 0.99
Loss: 0.12, Accuracy: 0.99
Loss: 0.11, Accuracy: 0.99
Loss: 0.10, Accuracy: 0.99
Loss: 0.09, Accuracy: 0.99
Loss: 0.08, Accuracy: 0.99
Loss: 0.08, Accuracy: 0.99
Loss: 0.07, Accuracy: 0.99
Loss: 0.06, Accuracy: 0.99
Loss: 0.06, Accuracy: 1.00
Loss: 0.06, Accuracy: 1.00


In [34]:
labels

tensor([7, 7, 7, 7, 7, 0, 7, 7, 7, 5, 3, 6, 7, 6, 7, 7], device='cuda:0',
       dtype=torch.int32)

In [103]:
torch.argmax(logits, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')